In [1]:
! pip install pyspark

In [2]:
from pyspark.sql import SparkSession

packages = [
    "org.apache.hadoop:hadoop-aws:3.2.0",
    "org.apache.spark:spark-avro_2.12:2.4.4",
    "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1",
    "org.postgresql:postgresql:42.2.18"

]

spark = SparkSession.builder.appName("extract data from event_store")\
    .master('spark://172.1.0.2:7077')\
    .config("spark.jars.packages", ",".join(packages))\
    .getOrCreate()

spark

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4f4c5405-7cd8-4e54-b549-014fd76a2f7f;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
	found org.apache.spark#spark-avro_2.12;2.4.4 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
	found org.postgresql#postgresql;42.2.18 in central
	found org.checkerframework#checker-qual;3.5.0 in centra

In [5]:
df_event_store = spark.read.format('mongo').option('spark.mongodb.input.uri','mongodb://172.1.0.10:27017/event_store.event_store').option("encoding","UTF-8").load()
df_event_store.show(5)

+--------------------+-----------------+------------------------+---------+--------------------+------------+
|                 _id|        amendDate|                    data|eventType|weddingCreatedAtDate| weddingDate|
+--------------------+-----------------+------------------------+---------+--------------------+------------+
|{61da87ad6350d75d...|1.641711532983E12|{1, 1, 1, 1, 婚紗攝影...|   DELETE|    1.64810377648E12|1.6926336E12|
|{61da87ad6350d75d...|1.641711532983E12|{1, 1, 1, 1, 婚紗攝影...|      ADD|    1.64810377648E12|1.6926336E12|
|{61da87ad6350d75d...|1.641711532983E12|{1, 1, 1, 2, 婚紗攝影...|   DELETE|    1.64810377648E12|1.6926336E12|
|{61da87ad6350d75d...|1.641711532983E12|{1, 1, 1, 2, 婚紗攝影...|      ADD|    1.64810377648E12|1.6926336E12|
|{61da87ad6350d75d...|1.641711532983E12|{1, 1, 1, 3, 婚禮當日...|   DELETE|    1.64810377648E12|1.6926336E12|
+--------------------+-----------------+------------------------+---------+--------------------+------------+
only showing top 5 rows



In [6]:
df_wedding_event = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://172.1.0.20:5432/wedding") \
    .option("dbtable", "wedding_event") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .load()
df_wedding_event.show(5)

+---+-----------------+------------+---+------+-----------------+---------+--------------------+--------------------+
| id|     wedding_name|wedding_date|pax|budget|banquet_vendor_id|church_id|          created_at|          updated_at|
+---+-----------------+------------+---+------+-----------------+---------+--------------------+--------------------+
|  1|  vocsajrem&suceg|  2023-08-22|106|287553|              173|      182|2022-03-24 14:36:...|2022-01-09 06:57:...|
|  2|   mifribcek&liki|  2023-05-27|109|307551|               25|      211|2022-03-22 17:34:...|2022-01-09 06:57:...|
|  3|   wofwet&bocajos|  2023-04-13|295|294273|              173|       37|2022-03-04 08:53:...|2022-01-09 06:57:...|
|  4|oretizuf&pukhizme|  2023-06-07|293|361174|               94|      192|2022-02-22 10:23:...|2022-01-09 06:57:...|
|  5|        tud&polba|  2022-10-23|144|330012|              110|       68|2022-03-16 23:21:...|2022-01-09 06:57:...|
+---+-----------------+------------+---+------+---------

In [9]:
df_event_store_budget_lower_range = df_event_store.join(df_wedding_event,df_wedding_event.id == df_event_store.data.wedding_event_id).select("id","amendDate","data","eventType","weddingCreatedAtDate","weddingDate","budget").where(df_wedding_event.budget <= 350000)

df_event_store_budget_upper_range = df_event_store.join(df_wedding_event,df_wedding_event.id == df_event_store.data.wedding_event_id).select("id","amendDate","data","eventType","weddingCreatedAtDate","weddingDate","budget").where(df_wedding_event.budget > 350000)

In [10]:
df_event_store_budget_lower_range.show(5)

+---+-----------------+------------------------+---------+--------------------+------------+------+
| id|        amendDate|                    data|eventType|weddingCreatedAtDate| weddingDate|budget|
+---+-----------------+------------------------+---------+--------------------+------------+------+
|  1|1.641711532983E12|{1, 1, 1, 1, 婚紗攝影...|   DELETE|    1.64810377648E12|1.6926336E12|287553|
|  1|1.641711532983E12|{1, 1, 1, 1, 婚紗攝影...|      ADD|    1.64810377648E12|1.6926336E12|287553|
|  1|1.641711532983E12|{1, 1, 1, 2, 婚紗攝影...|   DELETE|    1.64810377648E12|1.6926336E12|287553|
|  1|1.641711532983E12|{1, 1, 1, 2, 婚紗攝影...|      ADD|    1.64810377648E12|1.6926336E12|287553|
|  1|1.641711532983E12|{1, 1, 1, 3, 婚禮當日...|   DELETE|    1.64810377648E12|1.6926336E12|287553|
+---+-----------------+------------------------+---------+--------------------+------------+------+
only showing top 5 rows

